In [35]:
# Importo las librerías que voy a utilizar.

import pymongo
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
db = client["companies"]

from pandas.io.json import json_normalize

import pandas as pd

In [36]:
# Obtengo todo el dataset para visualizar bien los datos.

data = db.companies.find()
dataset = pd.DataFrame(list(data))
pd.set_option("display.max_columns", 500)
display(dataset.head())

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,NaN,NaN,1.0,Technology Platform Company,info@wetpaint.com,[{'external_url': 'http://www.geekwire.com/201...,17.0,10.0,2005.0,"[{'id': 888, 'round_code': 'a', 'source_url': ...",http://wetpaint-inc.com,"{'available_sizes': [[[150, 75], 'assets/image...",[],NaN,"[{'id': 5869, 'description': 'Wetpaint named i...",Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...",<p>Wetpaint is a technology platform company t...,[],abc2,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
1,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,NaN,NaN,NaN,2.0,Server Management Software,pr@adventnet.com,[],NaN,NaN,1996.0,[],http://adventnet.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,[],AdventNet,600.0,"[{'description': 'Headquarters', 'address1': '...","<p>AdventNet is now <a href=""/company/zoho-man...",[],abc3,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,NaN,NaN,NaN,3.0,Online Business Apps Suite,info@zohocorp.com,[{'external_url': 'http://www.online-tech-tips...,15.0,9.0,2005.0,[],http://zoho.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,"[{'id': 388, 'description': 'Zoho Reaches 2 Mi...",Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '...","<p>Zoho offers a suite of Business, Collaborat...",[],abc4,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,NaN,NaN,None,NaN,user driven social content website,feedback@digg.com,[{'external_url': 'http://www.sociableblog.com...,11.0,10.0,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",http://www.digg.com,"{'available_sizes': [[[150, 150], 'assets/imag...",[],None,"[{'id': 9588, 'description': 'Another Digg Exe...",Digg,60.0,"[{'description': None, 'address1': '135 Missis...",<p>Digg is a user driven social content websit...,[],digg,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bo

In [37]:
# Hago de nuevo la query, quedándome con lo que me interesa.

mydataset = db.companies.find({"$and": 
                               [{"offices.latitude": {"$exists": True}}, {"offices.latitude": {"$ne": None}},
                                {"offices.longitude": {"$exists": True}}, {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}}, {"category_code": {"$ne": None}},
                                {"number_of_employees": {"$exists": True}}, {"number_of_employees": {"$ne": None}},
                                {"founded_year": {"$exists": True}}, {"founded_year": {"$gte": 2008}},
                                {"deadpooled_year": None},
                                {"category_code": {"$ne": "other"}}, {"category_code": {"$ne": "semiconductor"}}, 
                                {"category_code": {"$ne": "finance"}}, {"category_code": {"$ne": "medical"}}, 
                                {"category_code": {"$ne": "health"}}, {"category_code": {"$ne": "manufacturing"}}, 
                                {"category_code": {"$ne": "real_estate"}}, {"category_code": {"$ne": "fashion"}}, 
                                {"category_code": {"$ne": "nanotech"}}, {"category_code": {"$ne": "hospitality"}}, 
                                {"category_code": {"$ne": "transportation"}}, {"category_code": {"$ne": "local"}}]}, 
                              {"_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "number_of_employees": 1, "offices.country_code": 1, "offices.latitude": 1, 
                               "offices.longitude": 1, "total_money_raised": 1})

In [38]:
# Normalizo la columna offices, para separar los subvalores en columnas.

mydataframe = json_normalize(data = mydataset, record_path = "offices", 
                             meta = ["category_code", "founded_year", "name",
                                    "number_of_employees", "total_money_raised"])

In [39]:
mydataframe.head(10)

,country_code,latitude,longitude,category_code,founded_year,name,number_of_employees,total_money_raised
0,USA,37.782263,-122.392142,software,2008,GoingOn,40,$8.5M
1,USA,42.358920,-71.057810,enterprise,2008,Cloudant,70,$18.3M
2,USA,47.597965,-122.151158,games_video,2008,YouBeQB,8,$0
3,USA,37.781265,-122.393229,games_video,2008,Crunchyroll,50,$4.8M
4,USA,40.757929,-73.985506,search,2012,PeekYou,20,$1.83M
5,USA,33.817100,-111.903500,mobile,2010,GENWI,25,$7.1M
6,USA,37.566879,-122.323895,web,2013,Fixya,30,$8M
7,DEU,53.707739,10.023246,games_video,2009,alluc,7,$0
8,CZE,49.225902,16.569282,network_hosting,2008,Webnode,30,$0
9,USA,34.031764,-118.461884,games_video,2008,WonderHowTo,12,$0


In [40]:
# Creo una columna que una las coordenadas. Creo tres tipos de columnas por si acaso necesito determinados formatos de
# valores más tarde: 1 con arrays de coordenadas y 2 con valores tipo geopoint, una en orden lon-lat y otra en lat-lon.

from geojson import Point

# Creo una lista uniendo las latitudes y longitudes.

coordinates_longlat = []

for i in range(len(mydataframe["latitude"])):
    coordinates_longlat.append((mydataframe["longitude"][i], mydataframe["latitude"][i]))
    
coordinates_latlong = []

for i in range(len(mydataframe["latitude"])):
    coordinates_latlong.append((mydataframe["latitude"][i], mydataframe["longitude"][i]))

In [41]:
# Actualizo las listas convirtiendo sus valores a puntos geoespaciales (no me dejaba hacerlo directamente arriba).
                               
coordinates_longlat2 = []

for i in coordinates_longlat:
    coordinates_longlat2.append(Point(i))
    
coordinates_latlong2 = []

for i in coordinates_latlong:
    coordinates_latlong2.append(Point(i))

In [42]:
# Creo columnas con los valores de las coordenadas en formato geoespacial y en array.

mydataframe["coordenadas"] = coordinates_longlat
mydataframe["coordinates_longlat"] = coordinates_longlat2
mydataframe["coordinates_latlong"] = coordinates_latlong2

In [43]:
mydataframe.head()

,country_code,latitude,longitude,category_code,founded_year,name,number_of_employees,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong
0,USA,37.782263,-122.392142,software,2008,GoingOn,40,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -..."
1,USA,42.358920,-71.057810,enterprise,2008,Cloudant,70,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7..."
2,USA,47.597965,-122.151158,games_video,2008,YouBeQB,8,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -..."
3,USA,37.781265,-122.393229,games_video,2008,Crunchyroll,50,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -..."
4,USA,40.757929,-73.985506,search,2012,PeekYou,20,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -..."


In [44]:
null_cols = mydataframe.isnull().sum()

null_cols[null_cols > 0]

Series([], dtype: int64)

In [45]:
# Categorizo en bins los valores de number_of_employees. Considero que, principalmente en la industria digital, el
# el tamaño de las compañías en cuanto a número de empleados se está reduciendo en gran medida. Empresas de enorme 
# valor pueden tener muy pocos empleados (Whatsapp tenía solo 55 con un valor de 19 billones de dólares). Por tanto, 
# en lugar de la categorización estándar de empresas según su tamaño (pequeña: 1-49, mediana:
# 50-249, grande: >=250) voy a personalizar las bins según mi criterio.
# Pongo los bins en valores numéricos para calcular más tarde su varianza, a la hora de analizar el ratio small:big

enterprize_size = [1, 2, 3]
cutoffs = [-1, 10, 99, float("inf")]

bins = pd.cut(mydataframe["number_of_employees"], cutoffs, labels = enterprize_size)
mydataframe["size(small=1, med=2, big=3)"] = bins
mydataframe = mydataframe.drop("number_of_employees", axis = 1)

In [46]:
mydataframe.head()

,country_code,latitude,longitude,category_code,founded_year,name,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong,"size(small=1, med=2, big=3)"
0,USA,37.782263,-122.392142,software,2008,GoingOn,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",2
1,USA,42.358920,-71.057810,enterprise,2008,Cloudant,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",2
2,USA,47.597965,-122.151158,games_video,2008,YouBeQB,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",1
3,USA,37.781265,-122.393229,games_video,2008,Crunchyroll,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",2
4,USA,40.757929,-73.985506,search,2012,PeekYou,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",2


In [47]:
# Categorizo también en bins los valores de category_code, según me interesen más o menos. 

mydataframe["category_code"].value_counts()

web                 365
software            142
games_video          94
advertising          90
mobile               79
ecommerce            78
consulting           74
enterprise           69
public_relations     49
search               34
network_hosting      32
social               14
security             10
analytics            10
hardware              9
cleantech             9
education             7
messaging             6
music                 5
news                  4
biotech               4
travel                4
design                4
legal                 2
sports                2
photo_video           1
Name: category_code, dtype: int64

In [48]:
sector_interest = {"software": 10, "games_video": 9, "ecommerce": 9, "analytics": 9, "hardware": 9, "design": 9, 
                   "biotech": 9, "music": 8, "web": 8, "legal": 8, "advertising": 8, "cleantech": 8, "messaging": 8, 
                   "news": 8, "mobile": 7, "sports": 7, "photo_video": 7, "consulting": 6, "security": 6, 
                   "enterprise": 5, "public_relations": 5, "search": 5, "network_hosting": 5, "social": 5, 
                   "education": 5, "travel": 5}

interest_sector = []

for i in range(len(mydataframe["category_code"])):
    interest_sector.append(sector_interest.get(mydataframe["category_code"][i]))
    
mydataframe["sector_interest"] = interest_sector

In [49]:
mydataframe.head()

,country_code,latitude,longitude,category_code,founded_year,name,total_money_raised,coordenadas,coordinates_longlat,coordinates_latlong,"size(small=1, med=2, big=3)",sector_interest
0,USA,37.782263,-122.392142,software,2008,GoingOn,$8.5M,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",2,10
1,USA,42.358920,-71.057810,enterprise,2008,Cloudant,$18.3M,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",2,5
2,USA,47.597965,-122.151158,games_video,2008,YouBeQB,$0,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",1,9
3,USA,37.781265,-122.393229,games_video,2008,Crunchyroll,$4.8M,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",2,9
4,USA,40.757929,-73.985506,search,2012,PeekYou,$1.83M,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",2,5


In [50]:
#filtered = mydataframe[mydataframe["coordenadas"] == (-122.3999719, 37.7870923)]
#filtered

In [51]:
# Tengo que convertir los valores de total_money_raised a la misma moneda.
# Primero creo una lista con los valores limpios.

money_raised = []

for i in range(len(mydataframe["total_money_raised"])):
    if mydataframe["total_money_raised"][i][-1] != "M":                # Si no son millones, no lo considero relevante.
        money_raised.append("0")
    elif mydataframe["total_money_raised"][i][:2] == "C$":        # Solo los dólares canadienses tienen dos caracteres.
        money_raised.append(str(mydataframe["total_money_raised"][i])[2:-1])
    else:
        money_raised.append(str(mydataframe["total_money_raised"][i])[1:-1])
        
money_raised[:10]

['8.5', '18.3', '0', '4.8', '1.83', '7.1', '8', '0', '0', '0']

In [52]:
# Redondeo para deshacerme de puntos, y así poder convertir los strings a enteros.

money_raised_clean = []

for i in money_raised:
    if "." in i:
        if int(i[i.index(".")+1]) >= 5:
            money_raised_clean.append(int(i[0:i.index(".")])+1)
        else:
            money_raised_clean.append(int(i[0:i.index(".")]))
    else:
        money_raised_clean.append(int(i))
        
money_raised_clean[:10]

[9, 18, 0, 5, 2, 7, 8, 0, 0, 0]

In [53]:
# Después creo una lista con la moneda de cada valor.

mycurrencies = []

for i in mydataframe["total_money_raised"]:
    mycurrencies.append(str(i)[0])
    
mycurrencies[:10]

['$', '$', '$', '$', '$', '$', '$', '$', '$', '$']

In [54]:
print(len(money_raised_clean))
print(len(mycurrencies))
print(len(mydataframe.index))

1197
1197
1197


In [55]:
mycurrencies_set = set(mycurrencies)
mycurrencies_set

{'$', 'C', '£', '€'}

In [56]:
# Utilizo una api para obtener los tipos de cambio actuales para las monedas de mis datos.

import json
import requests

url = "https://api.exchangeratesapi.io/latest?symbols=USD,GBP,CAD"

response = requests.get(url)
results = response.json()
exchanges = pd.DataFrame(results)

In [57]:
exchanges = exchanges.reset_index()

In [58]:
exchanges

,index,base,rates,date
0,CAD,EUR,1.5132,2019-05-10
1,GBP,EUR,0.8625,2019-05-10
2,USD,EUR,1.1230,2019-05-10


In [59]:
# Y finalmente creo una lista con los valores convertidos a euros.

monedas = {"$": "USD", "C": "CAD", "£": "GBP"}

money_raised_eur = []

for i in range(len(mycurrencies)):
    if mycurrencies[i] == "€":
        money_raised_eur.append(money_raised_clean[i])     # Los valores que ya están en euros no hay que convertirlos.
    else:
        for e in range(len(exchanges["index"])):
            if monedas.get(mycurrencies[i]) == exchanges["index"][e]:
                money_raised_eur.append(money_raised_clean[i]/exchanges["rates"][e]) 

In [60]:
# Creo una nueva columna a partir de esa lista, y elimino la anterior

mydataframe["money_raised_(M€)"] = money_raised_eur
mydataframe = mydataframe.drop("total_money_raised", axis = 1)

mydataframe.head()

,country_code,latitude,longitude,category_code,founded_year,name,coordenadas,coordinates_longlat,coordinates_latlong,"size(small=1, med=2, big=3)",sector_interest,money_raised_(M€)
0,USA,37.782263,-122.392142,software,2008,GoingOn,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",2,10,8.014248
1,USA,42.358920,-71.057810,enterprise,2008,Cloudant,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",2,5,16.028495
2,USA,47.597965,-122.151158,games_video,2008,YouBeQB,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",1,9,0.000000
3,USA,37.781265,-122.393229,games_video,2008,Crunchyroll,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",2,9,4.452360
4,USA,40.757929,-73.985506,search,2012,PeekYou,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",2,5,1.780944


In [61]:
null_cols = mydataframe.isnull().sum()

null_cols[null_cols > 0]

Series([], dtype: int64)

In [62]:
# Reordeno mis columnas para una mejor visualización.

columns = ["name", "category_code", "sector_interest", "country_code", "founded_year", "size(small=1, med=2, big=3)", 
           "money_raised_(M€)", "coordenadas", "coordinates_longlat", "coordinates_latlong", "latitude", "longitude"]

mydataframe = mydataframe.reindex(columns, axis= 1)

mydataframe.head()

,name,category_code,sector_interest,country_code,founded_year,"size(small=1, med=2, big=3)",money_raised_(M€),coordenadas,coordinates_longlat,coordinates_latlong,latitude,longitude
0,GoingOn,software,10,USA,2008,2,8.014248,"(-122.392142, 37.782263)","{'type': 'Point', 'coordinates': [-122.392142,...","{'type': 'Point', 'coordinates': [37.782263, -...",37.782263,-122.392142
1,Cloudant,enterprise,5,USA,2008,2,16.028495,"(-71.05781, 42.35892)","{'type': 'Point', 'coordinates': [-71.05781, 4...","{'type': 'Point', 'coordinates': [42.35892, -7...",42.358920,-71.057810
2,YouBeQB,games_video,9,USA,2008,1,0.000000,"(-122.151158, 47.597965)","{'type': 'Point', 'coordinates': [-122.151158,...","{'type': 'Point', 'coordinates': [47.597965, -...",47.597965,-122.151158
3,Crunchyroll,games_video,9,USA,2008,2,4.452360,"(-122.393229, 37.781265)","{'type': 'Point', 'coordinates': [-122.393229,...","{'type': 'Point', 'coordinates': [37.781265, -...",37.781265,-122.393229
4,PeekYou,search,5,USA,2012,2,1.780944,"(-73.985506, 40.757929)","{'type': 'Point', 'coordinates': [-73.985506, ...","{'type': 'Point', 'coordinates': [40.757929, -...",40.757929,-73.985506


In [63]:
# Exporto mi dataframe a json.

mydataframe.to_json("geospatial_clean.json", orient = "records", lines = True)